# Mountain car
小车在受重力影响的山谷中, 到达旗帜则算成功  
Q-Learning教学https://www.jiqizhixin.com/articles/2018-04-17-3

In [5]:
from IPython.display import display, HTML

# Create an HTML img tag to display the image with a set width and height
image_html = '<img src="../image/mountain_car.png" width="400" height="300" alt="Mountain Car"/>'

# Use IPython.display to show the image
display(HTML(image_html))

In [6]:
import gym
import numpy as np
import pandas as pd

env = gym.make("MountainCar-v0", render_mode="rgb_array")

# 有三种离散动作: 向左, 不动, 向右
print(f"动作: {env.action_space}")
# 观测中第一个元素是x轴坐标, 第二个元素是小车速度
print(f"x轴最大值和速度最大值: {env.observation_space.high}")
print(f"x轴最小值和速度最小值: {env.observation_space.low}")

CHUNK_SIZE = 20
DISCRETE_OBSERVATION_SPACE_SIZE = [CHUNK_SIZE] * len(env.observation_space.high)
print(f"离散观测结果集合为{DISCRETE_OBSERVATION_SPACE_SIZE}的二维数组")
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / DISCRETE_OBSERVATION_SPACE_SIZE
print(f"每两个临近状态之间的差值: {discrete_os_win_size}")

# 学习率 α
LEARNING_RATE = 0.1
# 衰退因子 γ
GAMMA = 0.95
# 贪心算法的ε
EPSILON = 0.1
# 训练周期
EPISODES = 500

动作: Discrete(3)
x轴最大值和速度最大值: [0.6  0.07]
x轴最小值和速度最小值: [-1.2  -0.07]
离散观测结果集合为[20, 20]的二维数组
每两个临近状态之间的差值: [0.09  0.007]


In [7]:
def get_discrete_state(state):
    discrete_state = ((state - env.observation_space.low) / discrete_os_win_size).astype(int)
    # 确保第一个元素(位置)的取值在 0 到 (DISCRETE_OBSERVATION_SPACE_SIZE - 1) 之间, 因为有时小车的惯性会超出边界
    discrete_state[0] = np.clip(discrete_state[0], 0, CHUNK_SIZE - 1)
    return tuple(discrete_state)

initial_observation, _ = env.reset(seed=42)
# 第一个元素为位置状态(共20个), 第二个元素为速度状态(共20个)
print(f"初始位置{initial_observation}")
discrete_state = get_discrete_state(initial_observation)
print(f"初始状态{discrete_state}")

初始位置[-0.4452088  0.       ]
初始状态(8, 10)


In [8]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder

# 初始q表, 随机填入-2到0之间的值
q_table_size = DISCRETE_OBSERVATION_SPACE_SIZE + [env.action_space.n]
q_table = np.random.uniform(low=-2, high=0, size=q_table_size)
print(f"Q表维度: {q_table.shape}")

# 刚开始非常慢, 第一回合或许会花费几分钟, 但是只要成功一次, 就会形成雪球效应越来越快
for ep in range(EPISODES):
    # 每回合都要重置
    terminated = False
    initial_observation, _ = env.reset(seed=42)
    discrete_state = get_discrete_state(initial_observation)
    # 记录开始时间
    step_count = 0
    # 在第?回合收集画面
    if ep in [0, 499]:
        render = True
        file_path = f"../video/mountain_car_ep{ep}.mp4"
        video = VideoRecorder(env, file_path)
    else:
        render = False
        video.close()
        env.close()

    while not terminated:
        if np.random.random() > EPSILON:
            # 0是向左, 1是不动, 2是向右
            action = np.argmax(q_table[discrete_state])
        # 有一定几率进行随机选择而非最优解, 用于探索, EPSILON越大则探索几率越大
        else:
            action = np.random.randint(0, env.action_space.n)
        new_state, reward, terminated, _, _ = env.step(action)
        new_discrete_state = get_discrete_state(new_state)
        if render:
            env.render()
            video.capture_frame()
        step_count += 1
        if not terminated:
            # 下一个状态的最大q值
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + (action, )]
            new_q = current_q + LEARNING_RATE * (reward + GAMMA * max_future_q - current_q)
            # 更新q表
            q_table[discrete_state + (action, )] = new_q
        # 到达目标地点, 奖励设置为0
        elif new_state[0] >= env.goal_position:
            print(f"在{ep}回合到达目标地点, 用了{step_count}步")
            q_table[discrete_state + (action, )] = 0
        discrete_state = new_discrete_state
video.close()
env.close()

Q表维度: (20, 20, 3)


/home/yuanhao/workspace/ai-manufacturing-demo/venv/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


在0回合到达目标地点, 用了12133步
Moviepy - Building video ../video/mountain_car_ep0.mp4.
Moviepy - Writing video ../video/mountain_car_ep0.mp4



Moviepy - Done !
Moviepy - video ready ../video/mountain_car_ep0.mp4
在1回合到达目标地点, 用了6108步
在2回合到达目标地点, 用了2521步
在3回合到达目标地点, 用了2122步
在4回合到达目标地点, 用了2696步
在5回合到达目标地点, 用了1655步
在6回合到达目标地点, 用了3934步
在7回合到达目标地点, 用了2808步
在8回合到达目标地点, 用了1657步
在9回合到达目标地点, 用了2429步
在10回合到达目标地点, 用了2815步
在11回合到达目标地点, 用了1747步
在12回合到达目标地点, 用了1119步
在13回合到达目标地点, 用了2093步
在14回合到达目标地点, 用了2642步
在15回合到达目标地点, 用了1092步
在16回合到达目标地点, 用了965步
在17回合到达目标地点, 用了5136步
在18回合到达目标地点, 用了1512步
在19回合到达目标地点, 用了739步
在20回合到达目标地点, 用了1280步
在21回合到达目标地点, 用了1118步
在22回合到达目标地点, 用了2606步
在23回合到达目标地点, 用了1063步
在24回合到达目标地点, 用了1451步
在25回合到达目标地点, 用了1393步
在26回合到达目标地点, 用了1404步
在27回合到达目标地点, 用了1354步
在28回合到达目标地点, 用了898步
在29回合到达目标地点, 用了1721步
在30回合到达目标地点, 用了820步
在31回合到达目标地点, 用了870步
在32回合到达目标地点, 用了1446步
在33回合到达目标地点, 用了1202步
在34回合到达目标地点, 用了944步
在35回合到达目标地点, 用了800步
在36回合到达目标地点, 用了1488步
在37回合到达目标地点, 用了1722步
在38回合到达目标地点, 用了774步
在39回合到达目标地点, 用了757步
在40回合到达目标地点, 用了649步
在41回合到达目标地点, 用了624步
在42回合到达目标地点, 用了858步
在43回合到达目标地点, 用了847步
在44回合到达目标地点, 用了658步
在45回合到达目标地点, 用了995步
在46回合到达目标地

Moviepy - Done !
Moviepy - video ready ../video/mountain_car_ep499.mp4
